# Χαρακτηρισμός δεδομένων με αυτόματο τρόπο

Στόχος είναι να χαρακτηρίσουμε με "έξυπνο" τρόπο το σύνολο των κειμένων με τις οντότητες που περιέχει το καθένα (και τις θέσεις αυτών στο κείμενο) ώστε να μπορέσουν να χρησιμοποιηθούν σε επόμενο στάδιο για την εκπαίδευση του Named Entity Recognizer. Αυτή η επισήμανση μπορεί να γίνει με αυτόματο τρόπο, πραγματοποιώντας αναζήτηση στα κείμενα με βάση γνωστές οντότητες που μας ενδιαφερούν στην παρούσα εργασία (βότανα, όργανα ανθρώπινου σώματος κλπ.).

Φτιάχνουμε 4 αρχεία txt, σε κάθενα από τα οποία βρίσκεται αποθηκευμένη μία λίστα με όρους που θα χρησιμοποιηθούν στην αναζήτηση. Στα αρχεία organs και plants βρίσκονται λέξεις που αποτελούν οντότητες των αντίστοιχων κατηγοριών ενώ στα αρχεία multiword_organs και multiword_plants βρίσκονται πάλι γνωστές οντότητες αυτών των κατηγοριών που αποτελούνται από παραπάνω από μία λέξεις.

Για να ταιριάξουμε τους όρους αναζήτησης στις λίστες που διαβάσαμε από τα txt, θα χρειαστεί διασπάσουμε τα κείμενα σε λέξεις (tokens). Αυτό γίνεται με τον tokenizer της spaCy για τα Ελληνικά. Επειδή όμως οι λέξεις μπορεί να βρίσκονται σε διάφορες πτώσεις πρέπει να κάνουμε lemmatization ή stemming τόσο στους όρους αναζήτησης όσο και στις λέξεις των κειμένων, ώστε να είναι επιτυχημένο το string matching που θα ακολουθήσει. 

Δοκιμάσαμε το lemmatizer της spaCy τόσο από το small όσο και από το medium model που διατίθενται για την ελληνική γλώσσα. Αν και το medium απέδωσε καλύτερα, γενικά η λημματοποίηση δεν είχε όσο ικανοποιητικά αποτελέσματα θα θέλαμε. Στα αποτελέσματα παρέμεναν διαφορετικές πτώσεις και πολλές φορές λανθασμένες παραποιήσεις της αρχικής λέξης, όπως θα δούμε στη συνέχεια. Για αυτό στραφήκαμε στη βιβλιοθήκη greek_stemmer που κάνει αποκατάληξη στα Ελληνικά, όπως λέει και το όνομά της. Συγκρίναμε τις δύο μεθόδους σε διάφορα παραδείγματα και όπως φαίνεται παρακάτω ο greek-stemmer αποδίδει καλύτερα και τα αποτελέσματά του φαίνονται να οδηγούν σε καλύτερο string matching αργότερα.

Και στις δύο μεθόδους παρατηρήθηκε ότι οι τόνοι μπορεί να οδηγούσαν σε διαφορετικά αποτελέσματα για την ίδια ρίζα, οπότε πρέπει να αφαιρεθούν προτού προχωρήσουμε με τη διαδικασία. Επίσης, η εναλλαγή πεζών κεφαλαίων κάποιες φορές οδήγησε σε διαφορετικά αποτελέσματα, οπότε επιλέχθηκε να μετατραπούν όλες οι λέξεις σε μία από τις δύο μορφές. Μάλιστα, ο greek-stemmer φαίνεται να λειτουργεί μόνο με κεφαλαία οπότε επιλέχθηκε αυτή η μετατροπή.

Οι μικρές διαφορές στις πτώσεις και στις καταλήξεις θα μπορούσαν να αντιμετωπιστούν εφαρμόζοντας fuzzy string matching, με υψηλό κατώφλι χαλάρωσης. Ωστόσο υπάρχουν περιπτώσεις όπου απαιτούνταν χαμηλότερο κατώφλι για να ταιριάξουν δυο λέξεις με κοινή ρίζα ενώ ένα πολύ ψηλό επαρκούσε για να ταιριάξουν δύο τελειώς διαφορετικές λέξεις, όπως φαίνεται και στο παρακάτω παράδειγμα. Τέτοια προβλήματα εξακολούθησαν να υπάρχουν, πιο σπάνια βέβαια, ακόμα και όταν συνδυάστηκε η λημματοποίηση του spaCy με fuzzy string matching.

Παρακάτω φαίνονται κάποια παραδείγματα που μας οδήγησαν σε αυτές τις αποφάσεις.

In [11]:
from fuzzywuzzy import process, fuzz
print(fuzz.ratio('έντερα', 'εντέρου'))
print(fuzz.ratio('καρδιά', 'καρυδιά'))

46
92


In [1]:
import spacy
import el_core_news_md
nlp = el_core_news_md.load()

In [2]:
from greek_stemmer import GreekStemmer
stemmer = GreekStemmer()

Παρακάτω παραθέτουμε κάποια παραδείγματα όπου φαίνεται η σύγκριση της λημματοποίησης με spaCy και της αποκατάληξης με greek_stemmer.

In [3]:
t = nlp('καρδια ΚΑΡΔΙΑ Καρδια καρδιας Καρδιας ΚΑΡΔΙΑΣ καρδιες ΚΑΡΔΙΕΣ καρδιων Καρδιων ΚΑΡΔΙΩΝ'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΚΑΡΔΙΑ καρδιο ΚΑΡΔ
ΚΑΡΔΙΑ καρδιο ΚΑΡΔ
ΚΑΡΔΙΑ καρδιο ΚΑΡΔ
ΚΑΡΔΙΑΣ καρδια ΚΑΡΔ
ΚΑΡΔΙΑΣ ΚΑΡΔΙΑΣ ΚΑΡΔ
ΚΑΡΔΙΑΣ ΚΑΡΔΙΑΣ ΚΑΡΔ
ΚΑΡΔΙΕΣ ΚΑΡΔΙΕΣ ΚΑΡΔ
ΚΑΡΔΙΕΣ ΚΑΡΔΙΕΣ ΚΑΡΔ
ΚΑΡΔΙΩΝ ΚΑΡΔΙΩΝ ΚΑΡΔ
ΚΑΡΔΙΩΝ καρδιων ΚΑΡΔ
ΚΑΡΔΙΩΝ καρδιων ΚΑΡΔ


In [4]:
t = nlp('εντερα Εντερα εντερων ΕΝΤΕΡΑ Εντερων εντερων ΕΝΤΕΡΩΝ εντερο Εντερο ΕΝΤΕΡΟ εντερου Εντερου ΕΝΤΕΡΟΥ εντερου'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΕΝΤΕΡΑ εντερα ΕΝΤΕΡ
ΕΝΤΕΡΑ εντερα ΕΝΤΕΡ
ΕΝΤΕΡΩΝ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΑ εντερα ΕΝΤΕΡ
ΕΝΤΕΡΩΝ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΩΝ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΩΝ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟΥ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟΥ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟΥ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟΥ εντερο ΕΝΤΕΡ


In [5]:
t = nlp('ουροποιητικο ουροποιητικου ουροποιητικου ουροποιητικου ΟΥΡΟΠΟΙΗΤΙΚΟΥ Ουροποιητικο'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΟΥΡΟΠΟΙΗΤΙΚΟ ουροποιητικος ΟΥΡΟΠΟΙΗΤ
ΟΥΡΟΠΟΙΗΤΙΚΟΥ ουροποιητικο ΟΥΡΟΠΟΙΗΤ
ΟΥΡΟΠΟΙΗΤΙΚΟΥ ουροποιητικο ΟΥΡΟΠΟΙΗΤ
ΟΥΡΟΠΟΙΗΤΙΚΟΥ ουροποιητικο ΟΥΡΟΠΟΙΗΤ
ΟΥΡΟΠΟΙΗΤΙΚΟΥ ουροποιητικο ΟΥΡΟΠΟΙΗΤ
ΟΥΡΟΠΟΙΗΤΙΚΟ ουροποιητικος ΟΥΡΟΠΟΙΗΤ


In [6]:
t = nlp('ενδοκρινεις Ενδοκρινεις Ενδοκρινων ΕΝΔΟΚΡΙΝΩΝ ενδοκρινων ενδοκρινης Ενδοκρινης ΕΝΔΟΚΡΙΝΗΣ'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΕΝΔΟΚΡΙΝΕΙΣ ενδοκρινεις ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΕΙΣ ενδοκρινεις ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΩΝ ενδοκρινος ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΩΝ ενδοκρινος ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΩΝ ενδοκρινος ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝ


In [7]:
t = nlp('ωοθηκες Ωοθηκες ΩΟΘΗΚΕΣ ωοθηκων ωοθηκη Ωοθηκων ΩΟΘΗΚΩΝ Ωοθηκη ωοθηκες ωοθηκες ωοθηκες ωοθηκες ωοθηκες'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΩΟΘΗΚΕΣ ωοθηκα ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκα ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκα ΩΟΘΗΚ
ΩΟΘΗΚΩΝ ωοθηκων ΩΟΘΗΚ
ΩΟΘΗΚΗ ωοθηκη ΩΟΘΗΚ
ΩΟΘΗΚΩΝ ωοθηκων ΩΟΘΗΚ
ΩΟΘΗΚΩΝ ωοθηκων ΩΟΘΗΚ
ΩΟΘΗΚΗ ωοθηκη ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκα ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκα ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκα ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκα ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκα ΩΟΘΗΚ


In [8]:
t = nlp('ρινικος ρινικη Ρινικη ΡΙΝΙΚΗ ρινικες ΡΙΝΙΚΕΣ ρινικες ρινικες ρινικες'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΡΙΝΙΚΟΣ ρινικος ΡΙΝ
ΡΙΝΙΚΗ ρινικη ΡΙΝ
ΡΙΝΙΚΗ ρινικη ΡΙΝ
ΡΙΝΙΚΗ ρινικη ΡΙΝ
ΡΙΝΙΚΕΣ ΡΙΝΙΚΕΣ ΡΙΝ
ΡΙΝΙΚΕΣ ΡΙΝΙΚΕΣ ΡΙΝ
ΡΙΝΙΚΕΣ ΡΙΝΙΚΕΣ ΡΙΝ
ΡΙΝΙΚΕΣ ΡΙΝΙΚΕΣ ΡΙΝ
ΡΙΝΙΚΕΣ ΡΙΝΙΚΕΣ ΡΙΝ


In [9]:
t = nlp('σμηγματογονος σμηγματογονοι Σμηγματαγονοι σμηγματογονων ΣΜΗΓΜΑΤΟΓΟΝΩΝ'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΣΜΗΓΜΑΤΟΓΟΝΟΣ ΣΜΗΓΜΑΤΟΓΟΝΟΣ ΣΜΗΓΜΑΤΟΓΟΝ
ΣΜΗΓΜΑΤΟΓΟΝΟΙ ΣΜΗΓΜΑΤΟΓΟΝΟΙ ΣΜΗΓΜΑΤΟΓΟΝ
ΣΜΗΓΜΑΤΑΓΟΝΟΙ σμηγματαγονοι ΣΜΗΓΜΑΤΑΓΟΝ
ΣΜΗΓΜΑΤΟΓΟΝΩΝ ΣΜΗΓΜΑΤΟΓΟΝΩΝ ΣΜΗΓΜΑΤΟΓΟΝ
ΣΜΗΓΜΑΤΟΓΟΝΩΝ ΣΜΗΓΜΑΤΟΓΟΝΩΝ ΣΜΗΓΜΑΤΟΓΟΝ


In [10]:
t = nlp('αγκιναρα αγκιναρες ΑΓΚΙΝΑΡΕΣ αγκιναρων Αγκιναρας'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΑΓΚΙΝΑΡΑ αγκιναρας ΑΓΚΙΝΑΡ
ΑΓΚΙΝΑΡΕΣ αγκιναρε ΑΓΚΙΝΑΡ
ΑΓΚΙΝΑΡΕΣ αγκιναρε ΑΓΚΙΝΑΡ
ΑΓΚΙΝΑΡΩΝ αγκιναρων ΑΓΚΙΝΑΡ
ΑΓΚΙΝΑΡΑΣ αγκιναρας ΑΓΚΙΝΑΡ


In [11]:
t = nlp('Ανηθος ανηθου ανηθο ΑΝΗΘΟ'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΑΝΗΘΟΣ ανηθος ΑΝΗΘ
ΑΝΗΘΟΥ ανηθο ΑΝΗΘ
ΑΝΗΘΟ ανηθο ΑΝΗΘ
ΑΝΗΘΟ ανηθο ΑΝΗΘ


In [12]:
t = nlp('Δαμασκηνια δαμασκηνα ΔΑΜΑΣΚΗΝΟ δαμασκηνων Δαμασκηνου δμασκηνιας'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΔΑΜΑΣΚΗΝΙΑ δαμασκηνια ΔΑΜΑΣΚΗΝ
ΔΑΜΑΣΚΗΝΑ δαμασκηνα ΔΑΜΑΣΚΗΝ
ΔΑΜΑΣΚΗΝΟ δαμασκηνο ΔΑΜΑΣΚΗΝ
ΔΑΜΑΣΚΗΝΩΝ δαμασκηνα ΔΑΜΑΣΚΗΝ
ΔΑΜΑΣΚΗΝΟΥ δαμασκηνου ΔΑΜΑΣΚΗΝ
ΔΜΑΣΚΗΝΙΑΣ δμασκηνιας ΔΜΑΣΚΗΝ


In [13]:
t = nlp('κρεμμυδι Κρεμμυδα κρεμμυδια κρεμμυδιων Κρεμμυδιου'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΚΡΕΜΜΥΔΙ ΚΡΕΜΜΥΔΙ ΚΡΕΜΜΥΔ
ΚΡΕΜΜΥΔΑ κρεμμυδας ΚΡΕΜΜΥΔ
ΚΡΕΜΜΥΔΙΑ κρεμμυδια ΚΡΕΜΜΥΔ
ΚΡΕΜΜΥΔΙΩΝ κρεμμυδιων ΚΡΕΜΜΥΔ
ΚΡΕΜΜΥΔΙΟΥ κρεμμυδιου ΚΡΕΜΜΥΔ


In [14]:
t = nlp('κυστη κυστης κυστεως κυστων κυστες κυστεις'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΚΥΣΤΗ κυστη ΚΥΣΤ
ΚΥΣΤΗΣ κυστη ΚΥΣΤ
ΚΥΣΤΕΩΣ κυστεως ΚΥΣΤ
ΚΥΣΤΩΝ ΚΥΣΤΩΝ ΚΥΣΤ
ΚΥΣΤΕΣ κυστε ΚΥΣΤ
ΚΥΣΤΕΙΣ κυστει ΚΥΣΤ


In [15]:
t = nlp('αδενας αδενων αδενα'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΑΔΕΝΑΣ αδενας ΑΔΕΝ
ΑΔΕΝΩΝ αδενο ΑΔΕΝ
ΑΔΕΝΑ αδενας ΑΔΕΝ


In [16]:
t = nlp('αγγελικη αγγελικης βασιλικος βασιλικου βασιλικο κοχλικο κοχλικου μηδικη μηδικης συναμικη συναμικης μυτερο μαςΤο μαστο'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΑΓΓΕΛΙΚΗ ΑΓΓΕΛΙΚΗ ΑΓΓΕΛ
ΑΓΓΕΛΙΚΗΣ ΑΓΓΕΛΙΚΗΣ ΑΓΓΕΛ
ΒΑΣΙΛΙΚΟΣ βασιλικος ΒΑΣΙΛ
ΒΑΣΙΛΙΚΟΥ βασιλικος ΒΑΣΙΛ
ΒΑΣΙΛΙΚΟ βασιλικος ΒΑΣΙΛ
ΚΟΧΛΙΚΟ ΚΟΧΛΙΚΟ ΚΟΧΛ
ΚΟΧΛΙΚΟΥ κοχλικου ΚΟΧΛ
ΜΗΔΙΚΗ μηδικη ΜΗΔ
ΜΗΔΙΚΗΣ μηδικης ΜΗΔ
ΣΥΝΑΜΙΚΗ συναμικη ΣΥΝΑΜ
ΣΥΝΑΜΙΚΗΣ συναμικης ΣΥΝΑΜ
ΜΥΤΕΡΟ μυτερο ΜΥΤ
ΜΑΣΤΟ μαστο ΜΑΣΤ
ΜΑΣΤΟ ΜΑΣΤΟ ΜΑΣΤ


In [17]:
t = nlp('μυες μυων μυουμαι μυηθει μυηση μυειται μυειτε μυουμε μυεις μυω μυς μυ'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΜΥΕΣ μυα ΜΥ
ΜΥΩΝ μυων ΜΥ
ΜΥΟΥΜΑΙ μυουμαι ΜΥ
ΜΥΗΘΕΙ ΜΥΗΘΕΙ ΜΥ
ΜΥΗΣΗ μυηση ΜΥΗΣ
ΜΥΕΙΤΑΙ μυειται ΜΥΕΙΤΑ
ΜΥΕΙΤΕ μυειτε ΜΥ
ΜΥΟΥΜΕ μυουμε ΜΥ
ΜΥΕΙΣ μυεις ΜΥ
ΜΥΩ μυω ΜΥ
ΜΥΣ μυς Μ
ΜΥ μυ ΜΥ


In [13]:
import pandas as pd
import numpy as np

import spacy
import el_core_news_md
nlp = el_core_news_md.load()
stop_words = nlp.Defaults.stop_words
stop_words |= {'.',',',';','?',':','!',' ','&','/','ή','-','(',')','[',']','{','}','"','"','`','~','\xa0','αυτώ'}
wrong_entities = ['πορτοκαλί','ρέικι','ίτα','μεα','μεις','μυρτω','μυρτώ','μυρτούς','τζιτζιφέ','σουρβώ','άρτεμις','ατρεμις',
                 'νηρέα','νηρέας','νηρεα','βαλσάμικο', 'κροκί','σιελογόνα','αμάραντα','γλώσσες','γλωσσών','πουρνάρα']

from greek_stemmer import GreekStemmer
stemmer = GreekStemmer()



def search_terms_from_txt(filename, encoding):
    """
    Reads the content of a txt file and produces a list of strings 
    -in our case a list of search terms- separated by comma.

    Parameters
    ----------
    filename : str
        the path (relative or absolute) of the txt file 
        which contains search terms
    encoding : str
        the encoding of the txt file which contains search terms

    Returns
    -------
    list
        a list of strings - search terms (mutliword or singleword)
    """
    with open(filename,'r',encoding=encoding) as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    for term in content:
        search_terms = term.split(',')
    return search_terms



def removing_accents(word):
    """
    Removes accents from a given word.
    
    Parameters
    ----------
    word : str
        the word from which we want to remove the accents

    Returns
    -------
    str
        the word without accents
    """
    return (word.replace('ά', 'α').replace('έ', 'ε').replace('ή', 'η').replace('ί', 'ι').replace('ό', 'ο').replace('ύ', 'υ')
            .replace('ώ', 'ω').replace('ϊ', 'ι').replace('ϋ', 'υ').replace('ΐ', 'ι').replace('ΰ', 'υ').replace('Ά', 'Α')
            .replace('Έ', 'Ε').replace('Ή', 'Η').replace('Ί', 'Ι').replace('Ό', 'Ο').replace('Ύ', 'Υ').replace('Ϊ', 'Ι')
            .replace('Ϋ', 'Υ'))



def expand_stop_words(stop_words_set):
    """
    Add stop words without accents.
    
    Parameters
    ----------
    stop_words_set : set
        the set of the already existing stop words

    Returns
    -------
    list
        a list that contains all the stopwords with or without accents
    """
    new_stop_words = [word for word in stop_words_set]
    new_stop_words += [removing_accents(word) for word in stop_words_set]
    return new_stop_words



def stemming(word):
    """
    Implements stemming for a given word.
    
    Parameters
    ----------
    word : str
        the word that is about to be stemmed

    Returns
    -------
    str
        the stem of the given word at uppercase letters
    """
    return stemmer.stem((removing_accents(word)).upper())



def multiword_terms_look_up_list(search_terms_list):
    """
    Given a list of search terms where each of them contains
    multiple words, produces a list of lists with the stems
    of their tokens.

    Parameters
    ----------
    search_terms_list : list
        the list that contains the multiword search terms

    Returns
    -------
    list
        a list of lists, each of them containing the stems of 
        the tokens of one multiword search term
    """
    look_up_list_parts = []
    for term in search_terms_list:
        stem_parts = [stemming(part) for part in term.split(' ')]
        look_up_list_parts +=[stem_parts]
    return look_up_list_parts



def singleword_terms_look_up_list(search_terms_list):
    """
    Given a list of singleword search terms, produces a list of 
    lists with their stems.

    Parameters
    ----------
    search_terms_list : list
        the list that contains the singleword search terms

    Returns
    -------
    list
        a list of the stems of the elements of the given list
    """
    return list(np.unique([stemming(term) for term in search_terms_list]))



def text_to_tokens_stems(text):
    """
    Given a text, produces a list of its stemmed tokens, excluding
    the stop words.

    Parameters
    ----------
    text : str
        the given text that is about to be tokenized

    Returns
    -------
    list
        a list of tuples that contain the tokens of the given text,
        the stemmed text of them (str), the start position (int) 
        and the end position (int) of them in te text and a boolean 
        variable which indicates whether the token has been already
        identified as a part of an entity or not
    """
    text2 = nlp(text)
    tokens_list = []
    # keep token, text of token, start and end index of token in text 
    # and whether it belongs to an entity or not
    for token in text2:
        tokens_list += [(token, token.text, token.idx, token.idx + len(token), False)]
    expanded_stopwords = expand_stop_words(stop_words)
    # remove stop words
    tokens = [(a,b,c,d,e) for (a,b,c,d,e) in tokens_list if not(a.is_stop or (a.lower_ in expanded_stopwords) 
                                                                or (a.lemma_ in expanded_stopwords))]
    # preparation and stemming of each token
    final_tokens = [(a, stemming(b), c, d,e) for (a,b,c,d,e) in tokens]
    return final_tokens



def check_iko_suffix(word, stem):
    """
    Given a word, checks whether it is an adjective
    or an existing entity.

    Parameters
    ----------
    word : str
        the word to be checked
        
    stem : str
        the stem of the given word

    Returns
    -------
    boolean
        True if it is a correct entinty and not an adjective.
    """
    return (not((word).endswith('ικός') or (word).endswith('ικού') or (word).endswith('ικό') or (word).endswith('ικέ') 
                or (word).endswith('ικοί') or (word).endswith('ικών') or (word).endswith('ικούς') or (word).endswith('ική') 
                or (word).endswith('ικής') or (word).endswith('ικές') or (word).endswith('ικά')) 
            or (stem == 'ΑΓΓΕΛ' or stem == 'ΒΑΣΙΛ' or stem == 'ΚΟΧΛ' or stem == 'ΜΗΔ' or stem == 'ΣΙΝΑΜ' or stem == 'ΣΥΝΑΜ'))



def wrong_entity(word):
    """
    Given a word, checks whether it is a correct entity or not.

    Parameters
    ----------
    word : str
        the word to be checked

    Returns
    -------
    boolean
        True if it is not an entinty.
    """
    word = word.lower()
    return (('λευκ' in word) or ('μυτερ' in word) or ('πιπερατ' in word) or (('αγν' in word) and not ('αγνι' in word))
           or (('αρν' in word) and not ('ίκη' in word)) or 
            (('βασίλ' in word) or (('βασιλ' in word) and not ('ικό' in word or 'ικο' in word or 'ικέ' in word or 'ικώ' in word)))
           or (('κασταν' in word) and not ('καστανι' in word)) 
           or ('κένταυρ' in word or ('κενταύρ' in word and not ('κενταύρι' in word))) or ('πνευμονί' in word)
           or ('βήχ' in word and not ('βήχι' in word)) or ('σπάρτη' in word) 
           or ('μαραθ' in word or ('μαράθ' in word and not ('μαράθο' in word or 'μαράθω' in word)))
           or ('φτερ' in word) or (('γαλ' in word) or ('γάλ' in word and not ('γάλι' in word)))
           or ('μήλι' in word) or ('γκουαράν' in word and not ('γκουαράνα' in word))
           or ('μάλλ' in word) or ('μπελλ' in word) or (('ματιά' in word) or ('ματιέ' in word)) or ('μαςτ' in word)
           or ('κυμίνη' in word) or ('άγγελ' in word or 'αγγέλ' in word or ('αγγελ' in word and not ('αγγελική' in word)))
           or ('μήδ' in word or ('μηδ' in word and not ('μηδική' in word)))
           or ('βρωμ' in word or ('βρώμ' in word and not ('βρώμη' in word)))
           or ('θάψ' in word and not ('θάψι' in word))
           or ('κάψ' in word)
           or ('ψύλλ' in word and not ('ψύλλι' in word))
           or (stemming(word) == 'ΜΥ' and not (word == 'μυ' or removing_accents(word) == 'μυες' 
                                               or removing_accents(word) == 'μυων'))
           or word in wrong_entities)



def annotation(input_csv_filename, known_entities_with_labels, output_csv_filename):
    """
    Annotates the texts in an input csv file with their entities.
    The entities are detected through a search process using 
    predefined already known entities and their labels. The 
    annotated data are stored in a new output csv file.

    Parameters
    ----------
    input_csv_file : str
        the path (relative or absolute) of the file that contains
        the texts
        
    known_entities_with_labels : list
        list of tuples each of them referring to a category of entities
        each tuple contains a path of a txt file with singleword
        search terms - known entities (str), a path of a txt file with
        multiword search terms - known entities (str) and the label of
        the category of entities that they belong (str)
        
    output_csv_filename : str
        the path (relative or absolute) of the output file where the
        annotated data will be stored        
    """
    df_texts = pd.read_csv(input_csv_filename)
    df_texts['entities'] = np.empty((len(df_texts), 0)).tolist()
    
    entities_lists = [(multiword_terms_look_up_list(search_terms_from_txt(multiword_list_filename, 'utf-8')), 
                       singleword_terms_look_up_list(search_terms_from_txt(singleword_list_filename, 'utf-8')), label) 
                      for (singleword_list_filename, multiword_list_filename, label) in known_entities_with_labels]

    total_singleword_entities = []
    # for each text
    for k in range(df_texts.shape[0]):
        entities = []
        text = df_texts.iloc[k]['text']
        #tokenization and stemming of the text
        text_tokens = text_to_tokens_stems(text)
        
        # firstly annotation of multiword entities 
        for (multiword_look_up_list, _, label) in entities_lists:
            for i in range(len(text_tokens)):
                (token, token_string, start_pos, end_pos, annotated) = text_tokens[i]
                for terms_list in multiword_look_up_list:
                    match = True
                    for j in range(len(terms_list)):
                        # string matching for each token in a multiword entity-search term
                        if (i+j) < len(text_tokens):
                            (_,b,_,next_end_pos,_) = text_tokens[i+j]
                            if b != terms_list[j]:
                                match = False
                                break
                        else:
                            match = False
                            break
                    if match:
                        for l in range(len(terms_list)):
                            token_list = list(text_tokens[i+l])
                            #set annotated=True for each token in the entity 
                            # to avoid double annotation as a singleword entity as well
                            token_list[4] = True     
                            text_tokens[i+l] = tuple(token_list)
                        entities += [(start_pos, next_end_pos, label)]
        
        # annotation of singleword entities 
        for (_, singleword_look_up_list, label) in entities_lists:
            for (token, token_string, start_pos, end_pos, annotated) in text_tokens:
                for term in singleword_look_up_list:
                    if (token_string == term and not annotated and len(token.text) > 1 
                        and check_iko_suffix(token.text, token_string) and not wrong_entity(token.text)):
                        if token_string == 'ΑΜΥΓΔΑΛ':
                            if ('λές' in token.text or 'λες' in token.text or 'ΛΕΣ' in token.text or 'λών' in token.text):
                                entities += [(start_pos, end_pos, 'ORGAN')]
                            else:
                                entities += [(start_pos, end_pos, 'PLANT')]
                        elif token_string == 'ΝΥΧ':
                            if ('άκι' in token.text or 'ακι' in token.text or 'ΑΚΙ' in token.text):
                                entities += [(start_pos, end_pos, 'PLANT')]
                            else:
                                entities += [(start_pos, end_pos, 'ORGAN')]
                        else:    
                            entities += [(start_pos, end_pos, label)]
                        total_singleword_entities += [token.text]

        df_texts.at[k,'entities'] = df_texts.iloc[k]['entities'] + entities
        
    df_texts.to_csv(output_csv_filename, index=False)
    return total_singleword_entities



def merge_two_annotated_datasets(input_filename1, input_filename2, output_filename):
    """
    It merges two annotated datasets stored in two csv files
    into one csv file.

    Parameters
    ----------
    input_filename1 : str
        the path of the first file that is about to be merged
        
    input_filename2 : str
        the path of the second file that is about to be merged
        
    output_filename : str
        the path of the output file that the merged dataset 
        will be stored
    """
    df1 = pd.read_csv(input_filename1)
    df2 = pd.read_csv(input_filename2)
    df_total = df1.append(df2, ignore_index=True)
    df_total.to_csv(output_filename, index=False)

In [14]:
ents = annotation('../data/proionta_tis_fisis_votana.csv',[('../data/organs.txt', '../data/multiword_organs.txt', 'ORGAN'),
                                                    ('../data/plants.txt', '../data/multiword_plants.txt', 'PLANT')], 
          '../data/proionta_tis_fisis_votana_annotated.csv')

In [15]:
print(list(np.unique(ents)))

['ASHWAGANDHA', 'Acmella', 'Alfalfa', 'Ashwagandha', 'Boswellia', 'Camellia', 'Canella', 'Capsicum', 'Chaparral', 'Chia', 'Cimicifuga', 'Cordyceps', 'GINSENG', 'GRAVIOLA', 'Galangal', 'Ginger', 'Ginkgo', 'Ginseng', 'Graviola', 'Gynostemma', 'Jiaogulan', 'Lapacho', 'MACA', 'Maca', 'Matcha', 'Nigella', 'Psyllium', 'alfalfa', 'ashwagandha', 'boswellia', 'capsicum', 'chaparral', 'chia', 'cordyceps', 'galangal', 'ginger', 'ginkgo', 'ginseng', 'gymnema', 'gynostemma', 'konjac', 'maca', 'matcha', 'nigella', 'µέντας', 'µήλα', 'µήλο', 'µανιτάρια', 'µπελαντόνα', 'ΆΜΠΕΛΟΣ', 'Άβρωνία', 'Άγνος', 'Άδωνις', 'Άκορο', 'Άκορος', 'Άμπελος', 'Άνηθος', 'Άνηθου', 'Άνθρίσκου', 'Άρκευθος', 'Άρνικα', 'Έλίχρυσο', 'Έντερα', 'Έντερο', 'Ήπαρ', 'Ήπατος', 'Ίλεξ', 'Ίνουλα', 'Ίριδα', 'Ίταμο', 'Ίταμος', 'Ύσσωπο', 'Ύσσωπος', 'ΑΓΚΙΝΑΡΑ', 'ΑΓΡΙΑΔΑ', 'ΑΓΡΙΜΟΝΙΟ', 'ΑΚΟΡΟΣ', 'ΑΛΘΑΙΑ', 'ΑΛΟΗ', 'ΑΛΧΗΜΙΛΛΗ', 'ΑΜΥΓΔΑΛΙΑ', 'ΑΝΗΘΟΣ', 'ΑΝΤΡΑΚΛΑ', 'ΑΠΗΓΑΝΟΣ', 'ΑΡΚΕΥΘΟΣ', 'ΑΡΠΑΓΟΦΥΤΟ', 'ΑΣΠΕΡΟΥΛΑ', 'ΑΥΤΙΑ', 'ΑΧΙΛΛΕΑ', 'Αβγαριά', 'Αβο

In [16]:
ents = annotation('../data/itrofi_votana.csv',[('../data/organs.txt', '../data/multiword_organs.txt', 'ORGAN'), 
                                               ('../data/plants.txt', '../data/multiword_plants.txt', 'PLANT')], 
                  '../data/itrofi_votana_annotated.csv')

In [17]:
print(list(np.unique(ents)))

['Alfalfa', 'Ashitaba', 'Ashwagandha', 'Capsicum', 'Chia', 'Cordyceps', 'Ginger', 'Ginseng', 'Maca', 'Psyllium', 'chia', 'ginkgo', 'ginseng', 'matcha', 'psyllium', 'Άνηθος', 'Άρνικα', 'Όρχι', 'Ύσσωπος', 'Αντράκλα', 'Αρτεμισία', 'Αψιθιάς', 'Βάλσαμα', 'Βαλεριάνα', 'Βαλσαμόχορτο', 'Βασιλικό', 'Βασιλικός', 'Βρώμη', 'Γαϊδουράγκαθο', 'Γλιστρίδα', 'Γλυκάνισος', 'Γλυκόριζα', 'Γλυκόριζας', 'Δενδρολίβανο', 'Δεντρολίβανο', 'Δυόσμος', 'Εστραγκόν', 'Θυμάρι', 'Κάνναβη', 'Κάρδαμο', 'Κανέλα', 'Κοκκινοπίπερο', 'Κολλιτσίδα', 'Κουρκουμάς', 'Κρίταμο', 'Κόλιανδρος', 'Κύμινο', 'Λεμόνι', 'Λουΐζα', 'Λουίζας', 'Μέντα', 'Μαντζουράνα', 'Μαϊντανός', 'Μελισσόχορτο', 'Μολόχα', 'Μυρώνι', 'Οστά', 'Παντζάρι', 'Παπαρουνόσπορος', 'Πικραλίδα', 'Πιπεριές', 'Ρίγανη', 'Ρίγανης', 'Σέννα', 'Σισάνδρα', 'Σκόρδο', 'Στέβια', 'Ταραξάκο', 'Τζίντζερ', 'Τριγωνέλλα', 'Τσίλι', 'Τσουκνίδα', 'Φασκόμηλο', 'Φιλύρα', 'Φτελιά', 'Χρένο', 'Ψύλλιουμ', 'άνηθο', 'άνηθος', 'άνηθου', 'άρνικα', 'έντερα', 'έντερο', 'ήπαρ', 'ήπατος', 'αβοκάντο', 'αγγε

In [18]:
merge_two_annotated_datasets('../data/proionta_tis_fisis_votana_annotated.csv', '../data/itrofi_votana_annotated.csv', 
                             '../data/votana_total_texts_annotated.csv')

Είναι σημαντικό πως πρέπει να κρατήσουμε τη θέση , βασικά την αρχή και το τέλος κάθε token. Αυτό είναι απαραίτητο, καθώς στα χαρακτηρισμένα δεδομένα πέρα από τις οντότητες που περιέχονται σε ένα κείμενο πρέπει να υπάχρουν και οι θέσεις εμφάνισης αυτών.

Επίσης, για να επιταχυνθεί η διαδικασία και να αποφευχθεί το ταίριασμα κάποιου όρου αναζήτησης με μία stop word, οι τελευταίες απορρίπτονται πριν αρχίσει η διαδικασία της αναζήτησης. Με αυτόν τον τρόπο γλιτώνουμε κάποιες άσκοπες συγκρίσεις.

Επίσης, φροντίζουμε να εντοπίσουμε πρώτα τις οντότητες που αποτελούνται από παραπάνω λέξεις και μετά θα επισημάνουμε τις υπόλοιπες εφόσον δεν ανήκουν σε κάποια από τις προηγούμενες. Αυτό γίνεται γιατί ως οντότητες υπάρχουν π.χ. τόσο η "χοληδόχος κύστη" όσο και η "χοληδόχος" σκέτο. Αν αναγνωριζόταν πρώτα η μονολεξική οντότητα, η άλλη δε θα εντοπιζόταν ποτέ.

Τέλος, για να λάβουμε καλύτερα αποτελέσματα δε λάβαμε υπόψη μας κάποιες οντότητες. Πιο συγκεκριμένα, αφαιρέθηκε ο θύμος αδένας από το σύστημα ενδοκρινών και το λεμφικό σύστημα, καθώς στα κείμενο ο θύμος συναντάται μόνο ως αρχαία ονομασία του θυμαριού. Ακόμα, οι σύνδεσμοι μόνο μία φορά εμφανίζονταν με την επιθυμητή σημασία που σχετίζεται με το ανθρώπινο σώμα και όλες τις άλλες με άλλη καθημερινή σημασία. Τέλος, από τα βότανα αφαιρέθηκε και η φούσκα, γιατί υπάρχει και στο ανθρώπινο σώμα.